In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
from cobra.io import write_sbml_model,save_json_model,load_json_model,read_sbml_model
import re
import pandas as pd
import numpy as np
import xlrd
import re
import openpyxl
import sys
sys.path.append(r'./')
import os
from copy import copy, deepcopy
import numpy as np

#### model json file

In [2]:
model_1515_773_file='./iCW773+iML1515/model/iML1515_iCW773.json'

In [3]:
model_origin_1515='./iCW773+iML1515/model/iML1515_new.json'
model_origin_773='./iCW773+iML1515/model/iCW773_final.json'

#### model construction

Sub-model A Construction(iML1515)

In [4]:
model1=load_json_model(model_origin_1515)
pydx5p_c=model1.metabolites.get_by_id('pydx5p_c')
lys__L_c=model1.metabolites.get_by_id('lys__L_c')
dap_c=model1.metabolites.get_by_id('15dap_c')
r5p_c=model1.metabolites.get_by_id('r5p_c')
g3p_c=model1.metabolites.get_by_id('g3p_c')
glu__L_c=model1.metabolites.get_by_id('glu__L_c')
gln__L_c=model1.metabolites.get_by_id('gln__L_c')
pi_c=model1.metabolites.get_by_id('pi_c')
h_c=model1.metabolites.get_by_id('h_c')
h2o_c=model1.metabolites.get_by_id('h2o_c')
#model1.reactions.get_by_id('EX_lys__L_e').bounds=(-1000,-10)
LYSDC=model1.reactions.get_by_id('LYSDC')
pdx=Reaction("pdx")
pdx.name='BioID:RXN-11322'
pdx.subsystem='unknow'
pdx.lower_bound=-1000
pdx.upper_bound=1000
pdx.add_metabolites({
    gln__L_c:-1.0,
    g3p_c:-1.0,
    r5p_c:-1.0,
    pydx5p_c:1.0,
    pi_c:1.0,
    glu__L_c:1.0,
    h2o_c:3.0,
    h_c:1.0
})
model1.add_reaction(pdx)
model1.reactions.get_by_id('EX_glc__D_e').bounds=(0,0)
# #model1.reactions.get_by_id('EX_glc__D_e_reverse').bounds=(0,0)
model1.reactions.get_by_id('EX_glyc_e').bounds=(-10,0)
model_origin_1515_final=model_origin_1515.split('.json')[0]+'_final.json'
save_json_model(model1,model_origin_1515_final)

In [5]:
# model1=load_json_model(model_origin_1515)
with model1 as model_test:
    print('Biomass:')
    model_test.optimize()
    model_test.summary()
    print('EX_15dap_e:')
    #model_test.reactions.pdx.bounds=(0,0)
    # model_test.reactions.get_by_id('EX_glyc_e').bounds=(-1000,0)
    # model_test.reactions.get_by_id('EX_glc__D_e').bounds=(-10,0)
    model_test.objective=model_test.reactions.EX_15dap_e
    model_test.optimize()
    model_test.summary()

Biomass:
IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  ----------------------
o2_e    13.6       h2o_e  32.7   BIOMASS_Ec_i...  0.497
glyc_e  10         co2_e   9.61
nh4_e    5.36      h_e     4.56
pi_e     0.479
so4_e    0.125
k_e      0.0969
fe2_e    0.00798
mg2_e    0.00431
ca2_e    0.00259
cl_e     0.00259
cu2_e    0.000352
mn2_e    0.000343
zn2_e    0.000169
ni2_e    0.00016
EX_15dap_e:
IN FLUXES      OUT FLUXES      OBJECTIVES
-------------  --------------  ----------------
glyc_e  10     h2o_e    22.3   EX_15dap_e  4.41
nh4_e    8.83  co2_e     7.93
o2_e     4.11  15dap_e   4.41


Sub-model B Construction(iCW773)

In [6]:
model2=load_json_model(model_origin_773)
glyc_c=model2.metabolites.get_by_id('glyc_c')
dha_c=model2.metabolites.get_by_id('dha_c')
dhap_c=model2.metabolites.get_by_id('dhap_c')
#ASPK reaction up
lys__L_c=model2.metabolites.get_by_id('lys__L_c')
EX_lys_L_e=model2.reactions.get_by_id('EX_lys_L_e')
EX_glyc_e=model2.reactions.get_by_id('EX_glyc_e')
# metabolites
nad_c=model2.metabolites.get_by_id('nad_c')
nadh_c=model2.metabolites.get_by_id('nadh_c')
h_c=model2.metabolites.get_by_id('h_c')
DhaD=Reaction("DhaD")
DhaD.name='BioID:GLYCDEH-RXN'
DhaD.subsystem='unknow'
DhaD.lower_bound=-1000
DhaD.upper_bound=1000
DhaD.add_metabolites({
    glyc_c:-1.0,
    nad_c:-1.0,
    dha_c:1.0,
    nadh_c:1.0,
    h_c:1.0
})
# metabolites
atp_c=model2.metabolites.get_by_id('atp_c')
adp_c=model2.metabolites.get_by_id('adp_c')
DhaK=Reaction("DhaK")
DhaK.name='BioID:GLYCERONE-KINASE-RXN'
DhaK.subsystem='unknow'
DhaK.lower_bound=0
DhaK.upper_bound=1000
DhaK.add_metabolites({
    dha_c:-1.0,
    atp_c:-1.0,
    dhap_c:1.0,
    adp_c:1.0,
    h_c:1.0,
})
model2.add_reactions([DhaD,DhaK])
EX_glyc_e.bounds=(-10,0)
EX_lys_L_e.bounds=(0,1000)
model2.reactions.get_by_id('EX_glc_e').bounds=(0,1000)
model2.reactions.get_by_id('EX_glc_e_reverse').bounds=(0,0)
model_origin_773_lys=model_origin_773.split('.json')[0]+'_lys.json'
save_json_model(model2,model_origin_773_lys)

In [7]:
with model2 as model_test:
    print('Biomass:')
    model_test.optimize()
    model_test.summary()
    print('EX_lys__L_e:')
    model_test.objective=model_test.reactions.EX_lys_L_e
    model_test.optimize()
    model_test.summary()

Biomass:
IN FLUXES            OUT FLUXES      OBJECTIVES
-------------------  --------------  ----------------------
o2_e       10        h2o_e  29       CG_biomass_c...  0.564
glyc_e     10        co2_e   6.65
nh4_e       4.88     h_e     4.35
k_e         0.714    xan_e   0.0905
pi_e        0.28
so4_e       0.0904
fe2_e       0.0531
mg2_e       0.0317
ca2_e       0.019
cl_e        0.019
cu2_e       0.00259
mn2_e       0.00253
zn2_e       0.00125
ni2_e       0.00118
cobalt2_e   9.2e-05
EX_lys__L_e:
IN FLUXES      OUT FLUXES       OBJECTIVES
-------------  ---------------  ----------------
glyc_e  10     h2o_e     21     EX_lys_L_e  4.74
nh4_e    9.48  lys__L_e   4.74
o2_e     1.82  h_e        4.74
               co2_e      1.56


medium analyze

construct double-model

In [8]:
model_a=load_json_model(model_origin_1515_final)
model_b=load_json_model(model_origin_773_lys)
# model_a.reactions.get_by_id('EX_glc__D_e').bounds=(-1000,1000)
# model_a.reactions.get_by_id('EX_'+meta_test+'_e').bounds=(0,1000)
# model_b.reactions.get_by_id('EX_'+meta_test+'_e').bounds=(-1000,0)
model_a.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)
model_b.reactions.get_by_id('EX_glc_e').bounds=(0,1000)
model_a.reactions.get_by_id('EX_glyc_e').bounds=(-1000,1000)
model_b.reactions.get_by_id('EX_glyc_e').bounds=(-1000,1000)
model_b.reactions.get_by_id('EX_lys_L_e').bounds=(-1000,1000)
model_a.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)
model_b.reactions.get_by_id('CG_biomass_cgl_ATCC13032').bounds=(0.1,1000)
print("--------A  model----------")
print(len(model_a.reactions))
print(len(model_a.metabolites))
print(len(model_a.genes) )
print("--------B  model----------")
print(len(model_b.reactions))
print(len(model_b.metabolites))
print(len(model_b.genes) )
model=deepcopy(model_a)
for i in model.reactions:
    i.id='A_'+i.id
for i in model.metabolites:
    i.id='A_'+i.id
    i.name='A_'+i.name
model_t=deepcopy(model_b)
# for i in model_t.genes:
#     i.id=i.id+'_B'
for i in model_t.reactions:
    i.id='B_'+i.id
for i in model_t.metabolites:
    i.id='B_'+i.id
    i.name='B_'+i.name
for i in model_t.reactions:
    model.add_reaction(i)
e_meta_rn={}
for meta in model_a.metabolites:
    if meta.compartment=='e' :
        meta_a=model.metabolites.get_by_id('A_'+meta.id)
        e_meta_rn[meta.id+"_1"]=Reaction('A_'+meta.id+'_con_1')
        e_meta_rn[meta.id+"_1"].name = meta.id+' Connection 1'
        e_meta_rn[meta.id+"_1"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_1"].lower_bound = 0. # This is the default
        e_meta_rn[meta.id+"_1"].upper_bound = 1000. # This is the default
        e_meta_rn[meta.id+"_1"].add_metabolites({
            meta_a:-1.0,
            meta:1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_1"]])
for meta in model_b.metabolites:
    if meta.compartment=='e' :
        meta_b=model_t.metabolites.get_by_id('B_'+meta.id)
        e_meta_rn[meta.id+"_2"]=Reaction('B_'+meta.id+'_con_2')
        e_meta_rn[meta.id+"_2"].name = meta.id+' Connection 2'
        e_meta_rn[meta.id+"_2"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_2"].lower_bound = -1000. # This is the default
        e_meta_rn[meta.id+"_2"].upper_bound = 0. # This is the default
        e_meta_rn[meta.id+"_2"].add_metabolites({
            meta_b:1.0,
            meta:-1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_2"]])
origin_model1=cobra.io.load_json_model(model_origin_1515_final)
origin_model1.medium
for i in origin_model1.medium:
    model.add_reaction(origin_model1.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    #model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('A_'+ex_meta+'_con_1')
origin_model2=cobra.io.load_json_model(model_origin_773_lys)
origin_model2.medium
for i in origin_model2.medium:
    r=model2.reactions.get_by_id(i)
    #print(r.metabolites.keys())
    for j in r.metabolites.keys():
        ex_meta=j.id#print(j)
    if 'EX_'+ex_meta not in model.medium:
        model.add_reaction(origin_model2.reactions.get_by_id(i))
    #model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('A_'+ex_meta+'_con_1')
# model.reactions.get_by_id('A_'+meta_test+'_e'+'_con_1').bounds=(-1000,1000)
# model.reactions.get_by_id('B_'+meta_test+'_e'+'_con_2').bounds=(-1000,1000)
for i in model.medium:
    h=i.split('_')[0]
    if h=='A' or h=='B':
        model.reactions.get_by_id(i).bounds=(0,0)
model.medium
#model.medium
EX_15dap_e=model1.reactions.get_by_id('EX_15dap_e')
model.add_reaction(EX_15dap_e)
save_json_model(model,model_1515_773_file)

--------A  model----------
2714
1877
1516
--------B  model----------
1852
949
794
A_pi_e_con_1
A_h_e_con_1
A_fe3_e_con_1
A_glyc_e_con_1
A_mn2_e_con_1
A_co2_e_con_1
A_fe2_e_con_1
A_zn2_e_con_1
A_mg2_e_con_1
A_ca2_e_con_1
A_ni2_e_con_1
A_cu2_e_con_1
A_cobalt2_e_con_1
A_sel_e_con_1
A_h2o_e_con_1
A_nh4_e_con_1
A_mobd_e_con_1
A_so4_e_con_1
A_k_e_con_1
A_na1_e_con_1
A_o2_e_con_1
A_cl_e_con_1
A_tungs_e_con_1
A_slnt_e_con_1
A_fe2_e_con_1
A_fe3_e_con_1
A_glyc_e_con_1
A_ca2_e_con_1
A_co2_e_con_1
A_cobalt2_e_con_1
A_cu2_e_con_1
A_h2o_e_con_1
A_k_e_con_1
A_mg2_e_con_1
A_mn2_e_con_1
A_na1_e_con_1
A_nh4_e_con_1
A_ni2_e_con_1
A_o2_e_con_1
A_pi_e_con_1
A_so4_e_con_1
A_zn2_e_con_1
A_btn_e_con_1
A_cl_e_con_1


In [9]:
model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glyc_e': 10,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_BIOTIN_reverse': 1000.0}

In [10]:
model.objective=model.reactions.get_by_id('EX_15dap_e')
model.optimize()
model.summary()

IN FLUXES          OUT FLUXES       OBJECTIVES
-----------------  ---------------  ----------------
glyc_e  10         B_h2o_e  26.1    EX_15dap_e  2.72
o2_e     7.41      B_co2_e   8.16
nh4_e    7.38      15dap_e   2.72
fe2_e    0.783     h_e       0.919
k_e      0.146     fe3_e     0.772
pi_e     0.146     B_xan_e   0.016
so4_e    0.0412
mg2_e    0.0065
ca2_e    0.0039
cl_e     0.0039
cu2_e    0.000531
mn2_e    0.000518
zn2_e    0.000255
ni2_e    0.000242


In [11]:
model.reactions.get_by_id('A_lys__L_e_con_1').bounds=(-1000,0)
model.reactions.get_by_id('B_lys__L_e_con_2').bounds=(-1000,0)


In [12]:
model.reactions.get_by_id('A_lys__L_e_con_1')

Reaction identifier,A_lys__L_e_con_1
Name,lys__L_e Connection 1
Memory address,0x02752dc05e48
Stoichiometry,A_lys__L_e <-- lys__L_e A_L-Lysine <-- L-Lysine
GPR,
Lower bound,-1000
Upper bound,0


In [13]:
model.reactions.get_by_id('B_lys__L_e_con_2')

Reaction identifier,B_lys__L_e_con_2
Name,lys__L_e Connection 2
Memory address,0x02752dddcd68
Stoichiometry,lys__L_e <-- B_lys__L_e L-Lysine <-- B_L-Lysine
GPR,
Lower bound,-1000
Upper bound,0


In [14]:
#lys__L_e Synthesis
model_1=load_json_model(model_origin_1515_final)
model.add_reaction(model_1.reactions.EX_lys__L_e)
save_json_model(model,model_1515_773_file)

In [15]:
model=load_json_model(model_1515_773_file)
model

Name,iML1515
Memory address,0x02752e8bb2b0
Number of metabolites,3183
Number of reactions,5146
Objective expression,1.0*EX_15dap_e - 1.0*EX_15dap_e_reverse_020d4
Compartments,"cytosol, extracellular space, periplasm"


In [16]:
model.reactions.EX_lys__L_e

Reaction identifier,EX_lys__L_e
Name,L-Lysine exchange
Memory address,0x027534f1c550
Stoichiometry,lys__L_e --> L-Lysine -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
model.objective=model.reactions.get_by_id('EX_15dap_e')
model.optimize()
model.summary()

IN FLUXES          OUT FLUXES       OBJECTIVES
-----------------  ---------------  ----------------
glyc_e  10         B_h2o_e  27.2    EX_15dap_e  2.78
o2_e     7.69      co2_e     5.91
nh4_e    7.5       fe3_e     3.55
fe2_e    3.56      15dap_e   2.78
h_e      1.86      B_co2_e   1.95
k_e      0.146     B_xan_e   0.016
pi_e     0.146
so4_e    0.0412
mg2_e    0.0065
ca2_e    0.0039
cl_e     0.0039
cu2_e    0.000531
mn2_e    0.000518
zn2_e    0.000255
ni2_e    0.000242


# Simulation

In [20]:
from CulECpyD_radio import *
from concurrent.futures import ProcessPoolExecutor, as_completed
import datetime
import sys
sys.path.append(r'./')

In [21]:
model_file=model_1515_773_file
reaction_kcat_MW_file1='./iCW773+iML1515/data/iML1515_reaction_kcat_MW.csv'
reaction_kcat_MW_file2='./iCW773+iML1515/data/iCW773_reaction_kcat_MW.csv'
reaction_kcat_MW_file3='./iCW773+iML1515/data/iML1515_iCW773_reaction_kcat_MW.csv'
Concretemodel_Need_Data=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file1,'None')

In [22]:
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw_diff_hosts(Concretemodel_Need_Data['reaction_list'],reaction_kcat_MW_file1,reaction_kcat_MW_file2,reaction_kcat_MW_file3)

Simulation Conditions

In [23]:
Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num1','kcat_MW']=Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num1','kcat_MW']*2

In [24]:
Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num2','kcat_MW']=Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num2','kcat_MW']*2

In [25]:
Concretemodel_Need_Data['lb_list']['EX_lys__L_e']=0.1

In [26]:
list_x=[1,1,1,2,3]
list_y=[3,2,1,1,1]

Simulated Results

In [27]:
if __name__ == "__main__":
    pool=ProcessPoolExecutor(max_workers=8)
    with pool as executor:
        start = datetime.datetime.now()
        #print("pass_2")
        tmp=[]
        futures = {executor.submit(diff_subs_iML_iCW,Concretemodel_Need_Data,(list_x[i],list_y[i]),{'EX_glyc_e_reverse':100},'EX_15dap_e'): i for i in range(0,len(list_x))}
        # for future in as_completed(futures):
        #     tmp.append(future.result())
    end = datetime.datetime.now()
    print (end - start)
    print(futures)

0:00:13.762233
{<Future at 0x27537758cc0 state=finished returned float>: 0, <Future at 0x2753775eba8 state=finished returned float>: 1, <Future at 0x2753775ec88 state=finished returned float>: 2, <Future at 0x2753775eda0 state=finished returned float>: 3, <Future at 0x2753775eef0 state=finished returned float>: 4}


In [28]:
result_list_mass=pd.DataFrame()
for future in as_completed(futures):
    x=list_x[futures[future]]
    y=list_y[futures[future]]
    real=1
    real_x=int(x*real)
    real_y=int(y*real)
    print('A:B='+str(real_x)+':'+str(real_y))
    print(future.result())
    result_list_mass.at[str(real_x)+':'+str(real_y),'flux']=future.result()
result_list=pd.DataFrame()
for i in range(0,len(list_x)):
    real=1
    real_x=int(list_x[i]*real)
    real_y=int(list_y[i]*real)
    result_list.at[str(real_x)+':'+str(real_y),'flux']=result_list_mass.at[str(real_x)+':'+str(real_y),'flux']
result_list

A:B=2:1
9.68769841161547
A:B=1:2
5.116174649405624
A:B=3:1
12.048507054125869
A:B=1:3
5.116188668789669
A:B=1:1
5.116174649405597


,flux
1:3,5.116189
1:2,5.116175
1:1,5.116175
2:1,9.687698
3:1,12.048507
